In [1]:
import py_entitymatching as em
import pandas as pd
import os,sys
import math

In [2]:
## Reading csv tables into pandas dataframe

A=em.read_csv_metadata('Data/csv/yelp_list.csv')
A['ID'] = range(0,len(A))
em.set_key(A,'ID')


B=em.read_csv_metadata('Data/csv/zomato_list.csv')
B['ID'] = range(0,len(B))
em.set_key(B,'ID')

True

In [3]:
##Data Cleaning1: Converting  price range to an absolute number. Will be applied for yelp list 


def clean_price_range( price_str ):

    avg_price = ''
    
    if not isinstance(price_str,basestring)  and  math.isnan(price_str):
        return price_str

    str2 = price_str.replace('$','')
        
    if '-' in str2:
        num_l = int(str2.split('-')[0])
        num_r = int(str2.split('-')[1])
        num_avg = (num_l+num_r)/2
        avg_price = '$' + str(num_avg)

    else:
        str_word = str2.split()[0].lower()
        num_price  = int(str2.split()[1])
        
        if str_word == 'above':
            num_avg = int(num_price + num_price/2)
            avg_price = '$' + str(num_avg)

        else:
            num_avg = num_price/2
            avg_price = '$' + str(num_avg)
            
    return avg_price

In [4]:
##Data Cleaning2: Removing "reviews" string from num_reviews column. Will be applied for zomato list


def clean_num_reviews(review_string) :
      return int(review_string.split()[0])

In [5]:
##Data Cleaning3: Removing "restaurant" if it is the last word in restaurant name. to prevent inconsistentcy.Will be applied
## for both lists


def clean_name( name_str):

    name_lower = name_str.lower()

    name_2 = name_lower.split()[-1]
    
    if name_2 == 'restaurant':
        name_lower = name_lower.replace('restaurant','')
        
    return name_lower

In [6]:
##Data Cleaning4: Removing quotes and shortening some words in address.Will be applied for both zomato list
## only zomato 

def clean_address( address ):
    addr_2 = address.replace('\"','')
    addr_3 = addr_2.replace(',','')
    if "Street" in addr_3:
        addr_3 = addr_3.replace('Street','St')
    if "Boulevard" in addr_3:
        addr_3 = addr_3.replace('Boulevard','Blvd')
        
    return addr_3

In [7]:
def clean_rating_value(ratingValue):
    return str(ratingValue)

In [8]:
########## DATA CLEANING ################

A['price_range'] = A['price_range'].apply( clean_price_range )
A['name'] = A['name'].apply( clean_name )
A['ratingValue'] = A['ratingValue'].apply(clean_rating_value)



B['name'] = B['name'].apply( clean_name )
B['address'] = B['address'].apply( clean_address )
B['number_of_reviews'] = B['number_of_reviews'].apply( clean_num_reviews )
B['ratingValue'] = B['ratingValue'].apply(clean_rating_value)

In [9]:
#To find missing values 
len(A.ID) - A.count()

number_of_reviews     0
price_range          44
ratingValue           0
name                  0
address               0
ID                    0
dtype: int64

In [10]:
len(B.ID) - B.count()

number_of_reviews    0
price_range          0
ratingValue          0
name                 0
address              0
ID                   0
dtype: int64

In [11]:
block_f = em.get_features_for_blocking(A,B)

In [12]:
block_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,number_of_reviews_number_of_reviews_exm,number_of_reviews,number_of_reviews,None,None,exact_match,<function number_of_reviews_number_of_reviews_exm at 0x7ff40ebbb6e0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,number_of_reviews_number_of_reviews_anm,number_of_reviews,number_of_reviews,None,None,abs_norm,<function number_of_reviews_number_of_reviews_anm at 0x7ff40ebbb5f0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,number_of_reviews_number_of_reviews_lev_dist,number_of_reviews,number_of_reviews,None,None,lev_dist,<function number_of_reviews_number_of_reviews_lev_dist at 0x7ff40ebbb8c0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,number_of_reviews_number_of_reviews_lev_sim,number_of_reviews,number_of_reviews,None,None,lev_sim,<function number_of_reviews_number_of_reviews_lev_sim at 0x7ff40ebbb578>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,price_range_price_range_lev_dist,price_range,price_range,None,None,lev_dist,<function price_range_price_range_lev_dist at 0x7ff40ebbb500>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,price_range_price_range_lev_sim,price_range,price_range,None,None,lev_sim,<function price_range_price_range_lev_sim at 0x7ff40ebbb488>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,price_range_price_range_jar,price_range,price_range,None,None,jaro,<function price_range_price_range_jar at 0x7ff40ebbb410>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,price_range_price_range_jwn,price_range,price_range,None,None,jaro_winkler,<function price_range_price_range_jwn at 0x7ff40e0e5050>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,price_range_price_range_exm,price_range,price_range,None,None,exact_match,<function price_range_price_range_exm at 0x7ff40e0e50c8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,price_range_price_range_jac_qgm_3_qgm_3,price_range,price_range,qgm_3,qgm_3,jaccard,<function price_range_price_range_jac_qgm_3_qgm_3 at 0x7ff40e0e5140>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [13]:
##Rule based on restaurant name similarity, 0.80 score is derived from debugging blocking output
rb1 = em.RuleBasedBlocker()
rule1 = 'name_name_lev_sim(ltuple,rtuple) < 0.60'
rb1.add_rule(rule1, block_f )

##Rule based on address similarity
rb2 = em.RuleBasedBlocker()
rule2 = 'address_address_jac_qgm_3_qgm_3(ltuple,rtuple) < 0.8'
rb2.add_rule(rule2,block_f)

'_rule_0'

In [14]:
## To block based on zipcode equivalence

def zipcode_match(x, y):
    # x, y will be of type pandas series
    
    # get address attribute
    x_address = x['address']
    y_address = y['address']
    
    # get the zipcode
    x_split, y_split = x_address.split(), y_address.split()
    x_zipcode = x_split[len(x_split) - 1]
    y_zipcode = y_split[len(y_split) - 1]
    
    # check if the zipcode match
    if x_zipcode != y_zipcode:
        return True
    else:
        return False

In [15]:
# Instantiate blackbox blocker
bb = em.BlackBoxBlocker()

# Set the black box function
bb.set_black_box_function(zipcode_match)

In [16]:
## Blocking Pipeline- First block based on zip code then block based on address similarity##

C1 = bb.block_tables(A,B,l_output_attrs=['ID','name','address','ratingValue','price_range','number_of_reviews'],r_output_attrs=['ID','name','address','ratingValue','price_range','number_of_reviews'],n_jobs=-1)

C2 = rb2.block_candset(C1,n_jobs=-1)

###Direct name blocker###
C3 = rb1.block_candset(C1,n_jobs=-1)

## Direct Address blocker not enabled##
#C2 = rb2.block_tables(A,B,l_output_attrs=['ID','name','address','ratingValue','price_range','number_of_reviews'],r_output_attrs=['ID','name','address','ratingValue','price_range','number_of_reviews'])

C4 = em.combine_blocker_outputs_via_union([C2, C3])

0%                          100%
[##############################] | ETA: 00:09:54 | ETA: 00:08:55 | ETA: 00:08:18 | ETA: 00:07:52 | ETA: 00:07:30 | ETA: 00:07:08 | ETA: 00:06:47 | ETA: 00:06:26 | ETA: 00:06:07 | ETA: 00:05:49 | ETA: 00:05:33 | ETA: 00:05:15 | ETA: 00:04:57 | ETA: 00:04:40 | ETA: 00:04:22 | ETA: 00:04:04 | ETA: 00:03:46 | ETA: 00:03:29 | ETA: 00:03:12 | ETA: 00:02:55 | ETA: 00:02:37 | ETA: 00:02:20 | ETA: 00:02:02 | ETA: 00:01:45 | ETA: 00:01:27 | ETA: 00:01:10 | ETA: 00:00:52 | ETA: 00:00:35 | ETA: 00:00:17 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:08:44
0%                          100%
[                              ]0%                          100%
[##############################] | ETA: 00:00:33 | ETA: 00:00:27 | ETA: 00:00:27 | ETA: 00:00:25 | ETA: 00:00:23 | ETA: 00:00:21 | ETA: 00:00:20 | ETA: 00:00:18 | ETA: 00:00:17 | ETA: 00:00:16 | ETA: 00:00:15 | ETA: 00:00:14 | ETA: 00:00:13 | ETA: 00:00:13 | ETA: 00:00:12 | ETA: 00:00:11 | ETA: 00:00:10 | ETA:

In [17]:
## Debugging blocker to make sure true positives are not left out in candidate set
dbg = em.debug_blocker(C4,A,B,output_size=400)

In [18]:
dbg

,_id,similarity,ltable_ID,rtable_ID,ltable_price_range,ltable_ratingValue,ltable_name,ltable_address,rtable_price_range,rtable_ratingValue,rtable_name,rtable_address
0,0,0.733333,2130,494,$20,4.0,stiles switch bbq & brew,6610 N Lamar Blvd Austin TX 78757,$40,3.8,stiles switch bbq & brew,6610 N Lamar Blvd Austin TX 78752
1,1,0.714286,2672,2576,$5,4.0,the sandwich spot,5126 Stevens Creek Blvd San Jose CA 95051,$35,2.8,the sandwich spot,5126 Stevens Creek Blvd San Jose CA 95129
2,2,0.692308,1584,1041,$45,3.5,bluefin japanese,754 The Alameda San Jose CA 95126,$80,3.5,bluefin,754 The Alameda St Suite 10 San Jose CA 95126
3,3,0.692308,3022,2933,$5,3.5,waffle house,7809 E Ben White Blvd Austin TX 78744,$15,3.2,waffle house,7809 E Ben White Blvd Austin TX 78741
4,4,0.687500,405,2726,$20,4.5,takai sushi and sake bar,170 S Market St San Jose CA 95113,$40,-,takai sushi & sake bar,170 S Market St San Jose CA
5,5,0.666667,604,2218,$20,4.0,cafe habana,17 Prince St New York NY 10012,$55,4.0,fanelli's cafe,94 Prince St New York NY 10012
6,6,0.666667,1486,2168,$20,3.0,affinity restaurant & bar,300 Almaden Blvd San Jose CA 95113,$35,-,affinity restaurant & bar,300 N Almaden Blvd San Jose CA 95110
7,7,0.666667,1679,851,$20,4.0,galaxy cafe,1000 W Lynn St Austin TX 78703,$40,4.0,zocalo cafe,1110 W Lynn St Austin TX 78703
8,8,0.666667,2876,461,$45,4.0,raymi,43 W 24th St New York NY 10010,$150,4.0,junoon,27 W 24th St New York NY 10010
9,9,0.666667,1810,3041,$5,4.5,agave mexican grill,17 S 4th St San Jose CA 95112,$30,-,el agave mexican grill,17 S 4th St San Jose CA 95113


In [19]:
##Writing final candidate set to CSV file
em.to_csv_metadata(C4,'./C4_output.csv')

True

In [22]:
S = em.sample_table(C4, 350)

In [23]:
G = em.label_table(S, 'gold_labels')

In [27]:
em.save_object(G, './GoldenData.pkl')

True

In [28]:
G = em.load_object('./GoldenData.pkl')

In [29]:
G.groupby('gold_labels').count()

,_id,ltable_ID,rtable_ID,ltable_name,ltable_address,ltable_ratingValue,ltable_price_range,ltable_number_of_reviews,rtable_name,rtable_address,rtable_ratingValue,rtable_price_range,rtable_number_of_reviews
gold_labels,,,,,,,,,,,,,
0,179,179,179,179,179,179,179,179,179,179,179,179,179
1,171,171,171,171,171,171,171,171,171,171,171,171,171


In [ ]:
train_test = em.split_train_test(G, train_proportion=0.5)
devel_set = train_test['train']

In [ ]:
eval_set = train_test['test']

In [ ]:
match_t = em.get_tokenizers_for_matching()
match_s = em.get_sim_funs_for_matching()

In [ ]:
atypes1 = em.get_attr_types(A) # don't need, if atypes1 exists from blocking step
atypes2 = em.get_attr_types(B) # don't need, if atypes2 exists from blocking step

match_c = em.get_attr_corres(A, B)

In [ ]:
match_f = em.get_features_for_matching(A, B)

In [ ]:
em._match_t

In [ ]:
em._match_s

In [ ]:
em._atypes1

In [37]:
em._atypes2

{'ID': 'numeric',
 '_table':      number_of_reviews price_range ratingValue                         name  \
 0                   24         $20         4.6         la victoria taqueria   
 1                   19         $40         4.6                  the counter   
 2                   44         $20         4.9           falafel's drive-in   
 3                   17         $40         4.4               new krung thai   
 4                   28         $70         4.1               original joe's   
 5                    6         $30         4.2                    adelita's   
 6                   14         $65         4.0      maggiano's little italy   
 7                   26         $40         4.4                blue rock bbq   
 8                   16         $43         4.2              zeni ethiopian    
 9                    9         $35         4.3              sam's bar-b-que   
 10                  14         $65         4.0                  amber india   
 11         

In [38]:
em._match_c

{'corres': [('number_of_reviews', 'number_of_reviews'),
  ('price_range', 'price_range'),
  ('ratingValue', 'ratingValue'),
  ('name', 'name'),
  ('address', 'address'),
  ('ID', 'ID')],
 'ltable':       number_of_reviews price_range  ratingValue  \
 0                   125         $20          5.0   
 1                  1041         $91          4.0   
 2                   146         $20          4.5   
 3                   266         $20          4.5   
 4                   159         $20          4.5   
 5                   287         $20          4.5   
 6                  1210         $20          4.5   
 7                   890         $20          4.0   
 8                   772         $45          4.5   
 9                   576         $20          4.5   
 10                  192         $45          4.0   
 11                  392         $20          4.0   
 12                  165         $20          4.5   
 13                  103         $20          5.0   
 14     

In [ ]:
H = em.extract_feature_vecs(G, match_f, attrs_before=['title'], attrs_after=['gold_labels'])

In [ ]:
H = em.extract_feature_vecs(G, features=match_f, attrs_before=['title'], attrs_after=['gold_labels'])

In [ ]:
H.fillna(value=0, inplace=True)

In [ ]:
dt = em.DTMatcher(max_depth=5)

In [ ]:
dt.fit(table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='gold_labels')

In [ ]:
dt.predict(table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='predicted_labels', append=True, inplace=True)

In [ ]:
train_test = em.split_train_test(devel, 0.5)

In [ ]:
train, test = train_test['train'], train_test['test']

In [ ]:
em.vis_debug_dt(dt, train, test, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='gold_labels')